In [1]:
import tensorflow as tf
for gpu in tf.config.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(gpu, True)

2022-09-05 11:58:41.622012: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-05 11:58:41.642790: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-05 11:58:41.642892: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
from brainseg.models import siamese
from tensorflow.keras import optimizers

from brainseg.handlers.siamese_histo_mri import (SiameseHistoMRIHandler2, list_descriptors,
                                                 extract_monkey_from_mri, list_descriptors_histo_only,
                                                 list_descriptors_mri_only,
                                                 SelfHistoMRIHandler)
from brainseg.provider import provider
from brainseg.loader import Loader
from brainseg.generator import X0TrainGenerator, XTestGenerator
import glob
import pandas as pd
import numpy as np

from random import sample, choices, choice
from math import floor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import imgaug.augmenters as iaa
from skimage.transform import resize

In [3]:
HISTO_LOCATION = "/media/moi/KINGSTON/BrainMapping/slides_512_self/*"

In [4]:
slides = glob.glob(HISTO_LOCATION)

In [5]:
desc = list_descriptors_histo_only(slides)

In [6]:
desc

[{'image_name': '/media/moi/KINGSTON/BrainMapping/slides_512_self/M155_LHRH141_CV.png',
  'monkey_name': 'M155',
  'slide_id': 141},
 {'image_name': '/media/moi/KINGSTON/BrainMapping/slides_512_self/M144-LH129-RH127.png',
  'monkey_name': 'M144',
  'slide_id': 127},
 {'image_name': '/media/moi/KINGSTON/BrainMapping/slides_512_self/M144-LH125-RH123.png',
  'monkey_name': 'M144',
  'slide_id': 123},
 {'image_name': '/media/moi/KINGSTON/BrainMapping/slides_512_self/M144-LH181-RH179.png',
  'monkey_name': 'M144',
  'slide_id': 179},
 {'image_name': '/media/moi/KINGSTON/BrainMapping/slides_512_self/M144-LH133-RH131.png',
  'monkey_name': 'M144',
  'slide_id': 131},
 {'image_name': '/media/moi/KINGSTON/BrainMapping/slides_512_self/M144-LH177-RH175.png',
  'monkey_name': 'M144',
  'slide_id': 175},
 {'image_name': '/media/moi/KINGSTON/BrainMapping/slides_512_self/M155_LHRH289_CV.png',
  'monkey_name': 'M155',
  'slide_id': 289},
 {'image_name': '/media/moi/KINGSTON/BrainMapping/slides_512_sel

## Setup enlargement for MRI images

In [7]:
import glob
import os

In [8]:
def get_oasis_id(path):
    return path.split("/")[-5]

def oasis_path_to_mask(path):
    return path.replace("t88_gfc", "t88_masked_gfc")

In [9]:
files = glob.glob("/home/moi/Downloads/oasis/*/*/*/PROCESSED/MPRAGE/T88_111/*t88_gfc.hdr")
len(files)

108

In [11]:
get_oasis_id(files[0])

'OAS1_0370_MR1'

In [12]:
oasis_path_to_mask(files[0])

'/home/moi/Downloads/oasis/oasis_cross-sectional_disc10/disc10/OAS1_0370_MR1/PROCESSED/MPRAGE/T88_111/OAS1_0370_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr'

In [13]:
import nibabel as nib
import matplotlib.pyplot as plt

In [14]:
desc_mri = list_descriptors_mri_only(files)
len(desc_mri)

very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:01.402 INFO    nibabel.global: very large origin values relative to dims; leaving as set, ignoring for affine
very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:01.403 INFO    nibabel.global: very large origin values relative to dims; leaving as set, ignoring for affine
very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:01.413 INFO    nibabel.global: very large origin values relative to dims; leaving as set, ignoring for affine
very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:01.414 INFO    nibabel.global: very large origin values relative to dims; leaving as set, ignoring for affine
very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:01.422 INFO    nibabel.global: very large origin values relative to dims; leavin

very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:01.605 INFO    nibabel.global: very large origin values relative to dims; leaving as set, ignoring for affine
very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:01.606 INFO    nibabel.global: very large origin values relative to dims; leaving as set, ignoring for affine
very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:01.616 INFO    nibabel.global: very large origin values relative to dims; leaving as set, ignoring for affine
very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:01.617 INFO    nibabel.global: very large origin values relative to dims; leaving as set, ignoring for affine
very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:01.627 INFO    nibabel.global: very large origin values relative to dims; leavin

very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:01.842 INFO    nibabel.global: very large origin values relative to dims; leaving as set, ignoring for affine
very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:01.843 INFO    nibabel.global: very large origin values relative to dims; leaving as set, ignoring for affine
very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:01.855 INFO    nibabel.global: very large origin values relative to dims; leaving as set, ignoring for affine
very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:01.857 INFO    nibabel.global: very large origin values relative to dims; leaving as set, ignoring for affine
very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:01.871 INFO    nibabel.global: very large origin values relative to dims; leavin

very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:02.104 INFO    nibabel.global: very large origin values relative to dims; leaving as set, ignoring for affine
very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:02.105 INFO    nibabel.global: very large origin values relative to dims; leaving as set, ignoring for affine
very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:02.120 INFO    nibabel.global: very large origin values relative to dims; leaving as set, ignoring for affine
very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:02.121 INFO    nibabel.global: very large origin values relative to dims; leaving as set, ignoring for affine
very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:02.135 INFO    nibabel.global: very large origin values relative to dims; leavin

very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:02.351 INFO    nibabel.global: very large origin values relative to dims; leaving as set, ignoring for affine
very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:02.352 INFO    nibabel.global: very large origin values relative to dims; leaving as set, ignoring for affine
very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:02.361 INFO    nibabel.global: very large origin values relative to dims; leaving as set, ignoring for affine
very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:02.362 INFO    nibabel.global: very large origin values relative to dims; leaving as set, ignoring for affine
very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:02.371 INFO    nibabel.global: very large origin values relative to dims; leavin

very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:02.552 INFO    nibabel.global: very large origin values relative to dims; leaving as set, ignoring for affine
very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:02.553 INFO    nibabel.global: very large origin values relative to dims; leaving as set, ignoring for affine
very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:02.562 INFO    nibabel.global: very large origin values relative to dims; leaving as set, ignoring for affine
very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:02.563 INFO    nibabel.global: very large origin values relative to dims; leaving as set, ignoring for affine
very large origin values relative to dims; leaving as set, ignoring for affine
2022-09-05 12:01:02.573 INFO    nibabel.global: very large origin values relative to dims; leavin

18144

In [15]:
desc_mri[:10]

[{'mri_name': '/home/moi/Downloads/oasis/oasis_cross-sectional_disc10/disc10/OAS1_0370_MR1/PROCESSED/MPRAGE/T88_111/OAS1_0370_MR1_mpr_n3_anon_111_t88_gfc.hdr',
  'slide_id': 20,
  'rotx': 0,
  'rotz': 0,
  'mri_id': 20,
  'has_mask': True},
 {'mri_name': '/home/moi/Downloads/oasis/oasis_cross-sectional_disc10/disc10/OAS1_0370_MR1/PROCESSED/MPRAGE/T88_111/OAS1_0370_MR1_mpr_n3_anon_111_t88_gfc.hdr',
  'slide_id': 21,
  'rotx': 0,
  'rotz': 0,
  'mri_id': 21,
  'has_mask': True},
 {'mri_name': '/home/moi/Downloads/oasis/oasis_cross-sectional_disc10/disc10/OAS1_0370_MR1/PROCESSED/MPRAGE/T88_111/OAS1_0370_MR1_mpr_n3_anon_111_t88_gfc.hdr',
  'slide_id': 22,
  'rotx': 0,
  'rotz': 0,
  'mri_id': 22,
  'has_mask': True},
 {'mri_name': '/home/moi/Downloads/oasis/oasis_cross-sectional_disc10/disc10/OAS1_0370_MR1/PROCESSED/MPRAGE/T88_111/OAS1_0370_MR1_mpr_n3_anon_111_t88_gfc.hdr',
  'slide_id': 23,
  'rotx': 0,
  'rotz': 0,
  'mri_id': 23,
  'has_mask': True},
 {'mri_name': '/home/moi/Downloads/o